In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import datetime
import pandas as pd
import pandasql as ps

In [ ]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
df =  pd.read_csv('./dict/category_subject.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

In [ ]:
# 불필요 컬럼 드롭
df.drop(['cate_n'], axis=1, inplace=True)

In [ ]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
third =  pd.read_csv('./final/third.txt', sep=',')
now = datetime.datetime.now()
print('종료:', now)

한번에 처리하려니 메모리 에러로 아래와같이 3개로 나누어 전처리

In [ ]:

# 청크 사이즈 정의
chunk_size = 50000000

# 처리 시작 시간
total_start_time = datetime.datetime.now()

# 청크 파일 저장을 위한 리스트
chunk_files = []

# 이미 읽어온 데이터프레임을 청크로 나누어 처리
total_length = len(df)
i = 0
for start in range(0, total_length, chunk_size):
    # 청크 처리 시작 시간
    chunk_start_time = datetime.datetime.now()
    
    end = min(start + chunk_size, total_length)
    # 남은 데이터 양 출력
    print(f"처리 중: {start+1} ~ {end} / {total_length}")
    
    # 청크 분할
    df_chunk = df.iloc[start:end]
    
    # 청크와 다른 데이터셋 조인
    #result_chunk = pd.merge(df_chunk, third_df, on='uid', how='inner')
    result_chunk = pd.merge(df_chunk, third, left_on='uid', right_on='refs_uid', how='inner')
    
    # 결과 저장 파일명 정의
    chunk_file = f'./mid/four/result_chunk_{i}.txt'
    chunk_files.append(chunk_file)

    # 청크 결과를 CSV 파일로 저장
    if i == 0:
        result_chunk.to_csv(chunk_file, mode='w', index=False, header=True)  # 첫 번째 파일에만 헤더 포함
    else:
        result_chunk.to_csv(chunk_file, mode='w', index=False, header=False)  # 나머지 파일에는 헤더 미포함
    
    # 청크 처리 완료 시간 및 소요 시간 출력
    chunk_end_time = datetime.datetime.now()
    chunk_duration = chunk_end_time - chunk_start_time
    print(f"청크 {i} 처리 완료: 소요 시간 = {chunk_duration}, 남은 데이터 양 = {total_length - end}")
    
    i += 1

# 처리 종료 시간 및 전체 소요 시간 출력
total_end_time = datetime.datetime.now()
total_duration = total_end_time - total_start_time
print(f"전체 처리 완료: 소요 시간 = {total_duration}")

# 병합된 파일 생성
#output_file = './mid/four/result_combined.txt'
#with open(output_file, 'w', newline='') as f_out:
#    for file in chunk_files:
#        with open(file, 'r', newline='') as f_in:
#            f_out.write(f_in.read())

처리 중: 1 ~ 50000000 / 138594301
청크 0 처리 완료: 소요 시간 = 0:29:30.027392, 남은 데이터 양 = 88594301
처리 중: 50000001 ~ 100000000 / 138594301
청크 1 처리 완료: 소요 시간 = 0:55:50.076194, 남은 데이터 양 = 38594301
처리 중: 100000001 ~ 138594301 / 138594301
청크 2 처리 완료: 소요 시간 = 0:21:12.056753, 남은 데이터 양 = 0
전체 처리 완료: 소요 시간 = 1:46:32.161339

In [ ]:
print(chunk_files )

['./mid/four/result_chunk_0.txt', './mid/four/result_chunk_1.txt', './mid/four/result_chunk_2.txt']

In [ ]:
# 병합된 파일 생성
#output_file = './mid/four/result_combined.txt'
#with open(output_file, 'w', newline='') as f_out:
#    for file in chunk_files:
#        with open(file, 'r', newline='') as f_in:
#            f_out.write(f_in.read())

In [ ]:
# reading 
#now = datetime.datetime.now() 
#print('시작:', now)
#df =  pd.read_csv('./mid/four/result_combined.txt', sep=',')
#now = datetime.datetime.now()
#print('종료:', now)

컴바인한 파일이 너무 커서 작업 불가 -> 모든 작업 후 합치는 방향으로 변경.

## 메모리 때문에 df1, df2, df3 각각 전처리 후 저장, 마지막에 최종파일로 병합.

In [2]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
df1 =  pd.read_csv('./mid/four/result_chunk_0.txt', sep=',', header=0)
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-12 09:21:51.122239
종료: 2024-03-12 09:27:26.004044


In [4]:
header = df1.columns.tolist()

In [3]:
df1

,uid_x,subject_traditional,uid_y,cluster_id,pubyear2,refs_uid,refs_n,within_core
0,WOS:000205970500001,Law,WOS:000345609800009,30,2014,WOS:000205970500001,73,0
1,WOS:000205970500001,Law,WOS:000434794200003,30,2018,WOS:000205970500001,90,0
2,WOS:000205970500001,Law,WOS:000528191500001,30,2020,WOS:000205970500001,150,0
3,WOS:000200960000105,Biology,WOS:000239862800001,911,2006,WOS:000200960000105,72,0
4,WOS:000188572100001,Geology,WOS:000653082100007,1351,2021,WOS:000188572100001,126,0
...,...,...,...,...,...,...,...,...
339005384,WOS:000166171000038,"Chemistry, Multidisciplinary",WOS:000270542600008,2367,2009,WOS:000166171000038,41,0
339005385,WOS:000166171000038,"Chemistry, Multidisciplinary",WOS:000257408200022,3252,2008,WOS:000166171000038,66,0
339005386,WOS:000166171000038,"Chemistry, Multidisciplinary",WOS:000286110900041,3308,2011,WOS:000166171000038,31,0
339005387,WOS:000170752800038,"Radiology, Nuclear Medicine & Medical Imaging",WOS:000329578200008,1749,2014,WOS:000170752800038,25,0


In [34]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
df2 =  pd.read_csv('./mid/four/result_chunk_1.txt', sep=',', header=None, names=header)
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-04 18:03:13.652444
종료: 2024-03-04 18:21:02.346182


In [21]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
df3 =  pd.read_csv('./mid/four/result_chunk_2.txt', sep=',', header=None, names=header)
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-04 17:27:10.576659
종료: 2024-03-04 17:30:37.215401


In [35]:
df2

,uid_x,subject_traditional,uid_y,cluster_id,pubyear2,refs_uid,refs_n,within_core
0,WOS:000174763600007,"Computer Science, Interdisciplinary Applications",WOS:000256953800052,90,2008,WOS:000174763600007,18,0
1,WOS:000174763600007,"Computer Science, Interdisciplinary Applications",WOS:000321259400013,229,2013,WOS:000174763600007,23,0
2,WOS:000174763600007,"Computer Science, Interdisciplinary Applications",WOS:000243370100007,1533,2006,WOS:000174763600007,13,0
3,WOS:000174763600007,"Computer Science, Interdisciplinary Applications",WOS:000274695000006,1533,2010,WOS:000174763600007,16,0
4,WOS:000174763600007,"Computer Science, Interdisciplinary Applications",WOS:000290678900006,1533,2011,WOS:000174763600007,16,0
...,...,...,...,...,...,...,...,...
831193222,WOS:000370657700012,"Engineering, Civil",WOS:000476764000001,1275,2019,WOS:000370657700012,45,0
831193223,WOS:000354256800005,Criminology & Penology,WOS:000577456700001,990,2020,WOS:000354256800005,53,0
831193224,WOS:000354256800005,Criminology & Penology,WOS:000712268900008,1584,2021,WOS:000354256800005,33,0
831193225,WOS:000354256800005,Criminology & Penology,WOS:000459858800001,3269,2019,WOS:000354256800005,76,0


In [22]:
df3

,uid_x,subject_traditional,uid_y,cluster_id,pubyear2,refs_uid,refs_n,within_core
0,WOS:000439202400008,Gastroenterology & Hepatology,WOS:000405694000104,8,2017,WOS:000439202400008,46,0
1,WOS:000439202400008,Gastroenterology & Hepatology,WOS:000447707900054,8,2018,WOS:000439202400008,40,0
2,WOS:000439202400008,Gastroenterology & Hepatology,WOS:000576146300001,8,2020,WOS:000439202400008,56,0
3,WOS:000439202400008,Gastroenterology & Hepatology,WOS:000530044900001,8,2020,WOS:000439202400008,60,0
4,WOS:000439202400008,Gastroenterology & Hepatology,WOS:000558676300014,8,2020,WOS:000439202400008,49,0
...,...,...,...,...,...,...,...,...
190156317,WOS:000784460300004,"Psychology, Multidisciplinary",WOS:000656941800001,326,2021,WOS:000784460300004,76,0
190156318,WOS:000784460300004,"Psychology, Multidisciplinary",WOS:000721372300007,1025,2021,WOS:000784460300004,85,0
190156319,WOS:000784460300004,"Psychology, Multidisciplinary",WOS:000713726300001,2713,2021,WOS:000784460300004,51,0
190156320,WOS:000813543000005,Law,WOS:000473282700004,3633,2019,WOS:000813543000005,47,0



최종 파일로 전처리 불가. 메모리 에러. 
아래와 같이 두개의 파일로 분할하여 작업 후 파일 병합.

df들을 컬럼 전처리(조인 컬럼 하나 삭제. uid_x(ref_uid)를 드롭, uid_y를 uid로 만들기.컬럼 순서변경. subject_traditinal을 cate로 변경. pubyear2를 pubyear로.

In [4]:
# 불필요 컬럼 드롭
df1.drop(['uid_x'], axis=1, inplace=True)
#df2.drop(['uid_x'], axis=1, inplace=True)
#df3.drop(['uid_x'], axis=1, inplace=True)
df1.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)
#df2.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)
#df3.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)



In [25]:
# 불필요 컬럼 드롭
#df1.drop(['uid_x'], axis=1, inplace=True)
#df2.drop(['uid_x'], axis=1, inplace=True)
df3.drop(['uid_x'], axis=1, inplace=True)
#df1.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)
#df2.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)
df3.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)



In [36]:
# 불필요 컬럼 드롭
#df1.drop(['uid_x'], axis=1, inplace=True)
df2.drop(['uid_x'], axis=1, inplace=True)
#df3.drop(['uid_x'], axis=1, inplace=True)
#df1.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)
df2.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)
#df3.rename(columns={'uid_y': 'uid', 'subject_traditional' : 'cate', 'pubyear2' : 'pubyear'}, inplace=True)



In [20]:
# 각 컬럼별 null 값의 개수 계산
null_counts22 =df2.isnull().sum()

print(null_counts22)

cate           0
uid            0
cluster_id     0
pubyear        0
refs_uid       0
refs_n         0
within_core    0
dtype: int64


In [21]:
# 각 컬럼별 null 값의 개수 계산
null_counts33 = df3.isnull().sum()

print(null_counts33)

cate           0
uid            0
cluster_id     0
pubyear        0
refs_uid       0
refs_n         0
within_core    0
dtype: int64


In [7]:
df1.rename(columns={'cate' : 'subject_traditional'},inplace=True)
#df2.rename(columns={'cate' : 'subject_traditional'},inplace=True)
#df3.rename(columns={'cate' : 'subject_traditional'},inplace=True)


In [26]:
#df1.rename(columns={'cate' : 'subject_traditional'},inplace=True)
#df2.rename(columns={'cate' : 'subject_traditional'},inplace=True)
df3.rename(columns={'cate' : 'subject_traditional'},inplace=True)


In [37]:
#df1.rename(columns={'cate' : 'subject_traditional'},inplace=True)
df2.rename(columns={'cate' : 'subject_traditional'},inplace=True)
#df3.rename(columns={'cate' : 'subject_traditional'},inplace=True)


cate_n생성

In [8]:
now = datetime.datetime.now() 
print('시작:', now)

# 'uid'별로 그룹화하여 'subject_traditional'의 고유한 개수를 계산
# 그리고 그 결과를 'cate_n'이라는 새로운 컬럼에 저장
df1['cate_n'] = df1.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())
#df2['cate_n'] = df2.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())
#df3['cate_n'] = df3.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-12 09:35:33.531208
종료: 2024-03-12 10:01:12.570042


In [27]:
now = datetime.datetime.now() 
print('시작:', now)

# 'uid'별로 그룹화하여 'subject_traditional'의 고유한 개수를 계산
# 그리고 그 결과를 'cate_n'이라는 새로운 컬럼에 저장
#df1['cate_n'] = df1.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())
#df2['cate_n'] = df2.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())
df3['cate_n'] = df3.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-04 17:32:55.066792
종료: 2024-03-04 17:54:01.297737


In [38]:
now = datetime.datetime.now() 
print('시작:', now)

# 'uid'별로 그룹화하여 'subject_traditional'의 고유한 개수를 계산
# 그리고 그 결과를 'cate_n'이라는 새로운 컬럼에 저장
#df1['cate_n'] = df1.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())
df2['cate_n'] = df2.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())
#df3['cate_n'] = df3.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-04 18:21:34.888415
종료: 2024-03-04 19:01:26.137820


In [9]:
df1

,subject_traditional,uid,cluster_id,pubyear,refs_uid,refs_n,within_core,cate_n
0,Law,WOS:000345609800009,30,2014,WOS:000205970500001,73,0,1
1,Law,WOS:000434794200003,30,2018,WOS:000205970500001,90,0,1
2,Law,WOS:000528191500001,30,2020,WOS:000205970500001,150,0,1
3,Biology,WOS:000239862800001,911,2006,WOS:000200960000105,72,0,1
4,Geology,WOS:000653082100007,1351,2021,WOS:000188572100001,126,0,1
...,...,...,...,...,...,...,...,...
339005384,"Chemistry, Multidisciplinary",WOS:000270542600008,2367,2009,WOS:000166171000038,41,0,1
339005385,"Chemistry, Multidisciplinary",WOS:000257408200022,3252,2008,WOS:000166171000038,66,0,1
339005386,"Chemistry, Multidisciplinary",WOS:000286110900041,3308,2011,WOS:000166171000038,31,0,1
339005387,"Radiology, Nuclear Medicine & Medical Imaging",WOS:000329578200008,1749,2014,WOS:000170752800038,25,0,1


In [10]:
unique_values = df1['cate_n'].value_counts()
print(unique_values)

cate_n
1    125599900
2    117996502
3     69969063
4     19337400
5      4290975
6      1749984
7        36526
8        21880
9         3159
Name: count, dtype: int64


In [39]:
# 유일성 체크: 'Article'과 'Review'의 빈도수 확인
unique_values22 = df2['cate_n'].value_counts()
print(unique_values22)

cate_n
2     274463326
1     261870914
3     187223655
4      65095172
6      24068580
5      18317325
7         58779
9         57366
8         37840
10          270
Name: count, dtype: int64


In [28]:
# 유일성 체크: 'Article'과 'Review'의 빈도수 확인
unique_values3 = df3['cate_n'].value_counts()
print(unique_values3)

cate_n
2    59587512
1    54905350
3    44785068
4    17636064
6     7765512
5     5458260
7       16954
8        1512
9          90
Name: count, dtype: int64


In [14]:
unique_values = df1['cate_n'].unique()
print(unique_values)


[1 2 3 4 5 6 8 9 7]


In [11]:
df1.rename(columns={'refs_n' : 'ref_n'},inplace=True)

In [29]:
df3.rename(columns={'refs_n' : 'ref_n'},inplace=True)

In [40]:
df2.rename(columns={'refs_n' : 'ref_n'},inplace=True)

In [12]:
#컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id',  'pubyear', 'ref_n', 'refs_uid', 'cate_n', 'subject_traditional', 'within_core']

# 새 순서로 DataFrame을 재구성
df1_reordered = df1[new_order]
#df2_reordered = df2[new_order]
#df3_reordered = df3[new_order]

In [30]:
#컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id',  'pubyear', 'ref_n', 'refs_uid', 'cate_n', 'subject_traditional', 'within_core']

# 새 순서로 DataFrame을 재구성
#df1_reordered = df1[new_order]
#df2_reordered = df2[new_order]
df3_reordered = df3[new_order]

In [41]:
#컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id',  'pubyear', 'ref_n', 'refs_uid', 'cate_n', 'subject_traditional', 'within_core']

# 새 순서로 DataFrame을 재구성
#df1_reordered = df1[new_order]
#df2_reordered = df2[new_order]
df2_reordered = df2[new_order]

In [13]:
df1_reordered

,uid,cluster_id,pubyear,ref_n,refs_uid,cate_n,subject_traditional,within_core
0,WOS:000345609800009,30,2014,73,WOS:000205970500001,1,Law,0
1,WOS:000434794200003,30,2018,90,WOS:000205970500001,1,Law,0
2,WOS:000528191500001,30,2020,150,WOS:000205970500001,1,Law,0
3,WOS:000239862800001,911,2006,72,WOS:000200960000105,1,Biology,0
4,WOS:000653082100007,1351,2021,126,WOS:000188572100001,1,Geology,0
...,...,...,...,...,...,...,...,...
339005384,WOS:000270542600008,2367,2009,41,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005385,WOS:000257408200022,3252,2008,66,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005386,WOS:000286110900041,3308,2011,31,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005387,WOS:000329578200008,1749,2014,25,WOS:000170752800038,1,"Radiology, Nuclear Medicine & Medical Imaging",0


In [31]:
df3_reordered

,uid,cluster_id,pubyear,ref_n,refs_uid,cate_n,subject_traditional,within_core
0,WOS:000405694000104,8,2017,46,WOS:000439202400008,1,Gastroenterology & Hepatology,0
1,WOS:000447707900054,8,2018,40,WOS:000439202400008,1,Gastroenterology & Hepatology,0
2,WOS:000576146300001,8,2020,56,WOS:000439202400008,1,Gastroenterology & Hepatology,0
3,WOS:000530044900001,8,2020,60,WOS:000439202400008,1,Gastroenterology & Hepatology,0
4,WOS:000558676300014,8,2020,49,WOS:000439202400008,1,Gastroenterology & Hepatology,0
...,...,...,...,...,...,...,...,...
190156317,WOS:000656941800001,326,2021,76,WOS:000784460300004,1,"Psychology, Multidisciplinary",0
190156318,WOS:000721372300007,1025,2021,85,WOS:000784460300004,1,"Psychology, Multidisciplinary",0
190156319,WOS:000713726300001,2713,2021,51,WOS:000784460300004,1,"Psychology, Multidisciplinary",0
190156320,WOS:000473282700004,3633,2019,47,WOS:000813543000005,1,Law,0


In [42]:
df2_reordered

,uid,cluster_id,pubyear,ref_n,refs_uid,cate_n,subject_traditional,within_core
0,WOS:000256953800052,90,2008,18,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
1,WOS:000321259400013,229,2013,23,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
2,WOS:000243370100007,1533,2006,13,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
3,WOS:000274695000006,1533,2010,16,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
4,WOS:000290678900006,1533,2011,16,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
...,...,...,...,...,...,...,...,...
831193222,WOS:000476764000001,1275,2019,45,WOS:000370657700012,1,"Engineering, Civil",0
831193223,WOS:000577456700001,990,2020,53,WOS:000354256800005,1,Criminology & Penology,0
831193224,WOS:000712268900008,1584,2021,33,WOS:000354256800005,1,Criminology & Penology,0
831193225,WOS:000459858800001,3269,2019,76,WOS:000354256800005,1,Criminology & Penology,0


within_core조사

In [14]:
now = datetime.datetime.now() 
print('시작:', now)
within_core_counts1 = df1_reordered['within_core'].value_counts()

# 결과 출력
print('df1')
print(within_core_counts1)

시작: 2024-03-12 10:01:25.195223
df1
within_core
0    339005389
Name: count, dtype: int64


시작: 2024-03-04 16:47:32.936105
df1
within_core
0    339005389
Name: count, dtype: int64

In [43]:
now = datetime.datetime.now() 
print('시작:', now)
within_core_counts222 = df2_reordered['within_core'].value_counts()

# 결과 출력
print('df2')
print(within_core_counts222)

시작: 2024-03-04 19:01:52.923431
df2
within_core
1    586450126
0    244743101
Name: count, dtype: int64


In [32]:
now = datetime.datetime.now() 
print('시작:', now)
within_core_counts33 = df3_reordered['within_core'].value_counts()

# 결과 출력
print('df3')
print(within_core_counts33)

시작: 2024-03-04 17:54:08.862430
df3
within_core
1    176351521
0     13804801
Name: count, dtype: int64


저장

In [15]:
df1_reordered.to_csv('./final/four/four_1.txt', sep='\t', index=False)

In [33]:
df3_reordered.to_csv('./final/four/four_3.txt', sep='\t', index=False)

In [44]:
df2_reordered.to_csv('./final/four/four_2.txt', sep='\t', index=False)

In [16]:
df1_reordered

,uid,cluster_id,pubyear,ref_n,refs_uid,cate_n,subject_traditional,within_core
0,WOS:000345609800009,30,2014,73,WOS:000205970500001,1,Law,0
1,WOS:000434794200003,30,2018,90,WOS:000205970500001,1,Law,0
2,WOS:000528191500001,30,2020,150,WOS:000205970500001,1,Law,0
3,WOS:000239862800001,911,2006,72,WOS:000200960000105,1,Biology,0
4,WOS:000653082100007,1351,2021,126,WOS:000188572100001,1,Geology,0
...,...,...,...,...,...,...,...,...
339005384,WOS:000270542600008,2367,2009,41,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005385,WOS:000257408200022,3252,2008,66,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005386,WOS:000286110900041,3308,2011,31,WOS:000166171000038,1,"Chemistry, Multidisciplinary",0
339005387,WOS:000329578200008,1749,2014,25,WOS:000170752800038,1,"Radiology, Nuclear Medicine & Medical Imaging",0


In [45]:
df2_reordered

,uid,cluster_id,pubyear,ref_n,refs_uid,cate_n,subject_traditional,within_core
0,WOS:000256953800052,90,2008,18,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
1,WOS:000321259400013,229,2013,23,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
2,WOS:000243370100007,1533,2006,13,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
3,WOS:000274695000006,1533,2010,16,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
4,WOS:000290678900006,1533,2011,16,WOS:000174763600007,3,"Computer Science, Interdisciplinary Applications",0
...,...,...,...,...,...,...,...,...
831193222,WOS:000476764000001,1275,2019,45,WOS:000370657700012,1,"Engineering, Civil",0
831193223,WOS:000577456700001,990,2020,53,WOS:000354256800005,1,Criminology & Penology,0
831193224,WOS:000712268900008,1584,2021,33,WOS:000354256800005,1,Criminology & Penology,0
831193225,WOS:000459858800001,3269,2019,76,WOS:000354256800005,1,Criminology & Penology,0


## 0, 1, 2 불러와서 subject_traditional 전처리   
각 df 카테고리 개수 합집합하여 254개 나오는지 검증  
각각 저장도 하기  
최종결과물로 파일 병합하여 저장  


In [18]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
four_1 = pd.read_csv('./final/four/four_1.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-12 10:52:33.270530
종료: 2024-03-12 10:57:49.117228


In [19]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
four_2 = pd.read_csv('./final/four/four_2.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-12 10:57:49.122272
종료: 2024-03-12 11:13:15.833237


In [20]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
four_3 = pd.read_csv('./final/four/four_3.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-12 11:13:15.837504
종료: 2024-03-12 11:16:33.683934


GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY  
→  
Green & Sustainable Science & Technoology 

In [27]:
# subject_traditional이 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY'인 행 추출
result_df_1_1 = four_1[four_1['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY']
result_df_1_2 = four_1[four_1['subject_traditional'] == 'Green & Sustainable Science & Technology']
print(len(result_df_1_1))
print(len(result_df_1_2))

16
38133


In [29]:
# subject_traditional이 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY'인 행 추출
result_df_2_1 = four_2[four_2['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY']
result_df_2_2 = four_2[four_2['subject_traditional'] == 'Green & Sustainable Science & Technology']
print(len(result_df_2_1))
print(len(result_df_2_2))

2
1671929


In [30]:
# subject_traditional이 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY'인 행 추출
result_df_3_1 = four_3[four_3['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY']
result_df_3_2 = four_3[four_3['subject_traditional'] == 'Green & Sustainable Science & Technology']
print(len(result_df_3_1))
print(len(result_df_3_2))

1
1592784


WOS Category 전처리

In [32]:
# 'subject_traditional' 컬럼 값 변경
four_1.loc[four_1['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY', 'subject_traditional'] = 'Green & Sustainable Science & Technology'

In [33]:
# 'subject_traditional' 컬럼 값 변경
four_2.loc[four_2['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY', 'subject_traditional'] = 'Green & Sustainable Science & Technology'

In [34]:
# 'subject_traditional' 컬럼 값 변경
four_3.loc[four_3['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY', 'subject_traditional'] = 'Green & Sustainable Science & Technology'

In [35]:
# subject_traditional이 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY'인 행 추출
result_df_1_1 = four_1[four_1['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY']
result_df_1_2 = four_1[four_1['subject_traditional'] == 'Green & Sustainable Science & Technology']
print(len(result_df_1_1))
print(len(result_df_1_2))

0
38149


In [36]:
# subject_traditional이 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY'인 행 추출
result_df_2_1 = four_2[four_2['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY']
result_df_2_2 = four_2[four_2['subject_traditional'] == 'Green & Sustainable Science & Technology']
print(len(result_df_2_1))
print(len(result_df_2_2))

0
1671931


In [37]:
# subject_traditional이 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY'인 행 추출
result_df_3_1 = four_3[four_3['subject_traditional'] == 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY']
result_df_3_2 = four_3[four_3['subject_traditional'] == 'Green & Sustainable Science & Technology']
print(len(result_df_3_1))
print(len(result_df_3_2))

0
1592785


Category 개수 조사

In [38]:
now = datetime.datetime.now() 
print('시작:', now)

# 'subject_traditional' 컬럼의 고유한 값들을 찾습니다.


unique_subjects1 = four_1['subject_traditional'].unique()
unique_subjects2 = four_2['subject_traditional'].unique()
unique_subjects3 = four_3['subject_traditional'].unique()


# 'subject_traditional' 컬럼의 고유한 값들로 이루어진 세트 생성
set_unique_subjects1 = set(unique_subjects1)
set_unique_subjects2 = set(unique_subjects2)
set_unique_subjects3 = set(unique_subjects3)
print('num_set_df1 : ')
print(len(set_unique_subjects1))
print('num_set_df1 : ')
print(len(set_unique_subjects2))
print('num_set_df1 : ')
print(len(set_unique_subjects3))


# 세 세트의 합집합 구하기
union_set = set_unique_subjects1 | set_unique_subjects2 | set_unique_subjects3


# 합집합의 요소 개수 세기
union_count = len(union_set)



# 254개인지 확인
if union_count == 254:
    print("카테고리 개수가 정확히 254개입니다.")
else:
    print(f"카테고리 개수가 예상과 다릅니다: {union_count}개")
    
    
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-12 13:34:49.649766
num_set_df1 : 
255
num_set_df1 : 
254
num_set_df1 : 
254
카테고리 개수가 예상과 다릅니다: 255개
종료: 2024-03-12 13:36:10.809376


In [40]:
print(set_unique_subjects1 - set_unique_subjects2)
print(set_unique_subjects1 - set_unique_subjects3)

{'Planning & Development'}
{'Planning & Development'}


In [41]:
import pickle
# Pickle 파일에서 리스트 불러오기
with open('./mid/unique_subjects.pkl', 'rb') as file:
    loaded_unique_subjects = pickle.load(file)

In [45]:
loaded_unique_subjects = set(loaded_unique_subjects)
print(set_unique_subjects1 - loaded_unique_subjects)

{'Planning & Development'}


In [44]:
print(len(loaded_unique_subjects))

254


WOS Category에 속하지 않는{'Planning & Development'} 행은 드롭.

In [47]:
# subject_traditional이 'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY'인 행 추출
result1 = four_1[four_1['subject_traditional'] == 'Planning & Development']
result2 = four_2[four_2['subject_traditional'] == 'Planning & Development']
result3 = four_3[four_3['subject_traditional'] == 'Planning & Development']

print(len(result1))
print(len(result2))
print(len(result3))

5
0
0


In [48]:
# 'subject_traditional' 컬럼 값이 'Planning & Development'인 행 드롭
four_1 = four_1.drop(four_1[four_1['subject_traditional'] == 'Planning & Development'].index)

In [49]:
now = datetime.datetime.now() 
print('시작:', now)

# 'subject_traditional' 컬럼의 고유한 값들을 찾습니다.


unique_subjects1 = four_1['subject_traditional'].unique()
unique_subjects2 = four_2['subject_traditional'].unique()
unique_subjects3 = four_3['subject_traditional'].unique()


# 'subject_traditional' 컬럼의 고유한 값들로 이루어진 세트 생성
set_unique_subjects1 = set(unique_subjects1)
set_unique_subjects2 = set(unique_subjects2)
set_unique_subjects3 = set(unique_subjects3)
print('num_set_df1 : ')
print(len(set_unique_subjects1))
print('num_set_df1 : ')
print(len(set_unique_subjects2))
print('num_set_df1 : ')
print(len(set_unique_subjects3))


# 세 세트의 합집합 구하기
union_set = set_unique_subjects1 | set_unique_subjects2 | set_unique_subjects3


# 합집합의 요소 개수 세기
union_count = len(union_set)



# 254개인지 확인
if union_count == 254:
    print("카테고리 개수가 정확히 254개입니다.")
else:
    print(f"카테고리 개수가 예상과 다릅니다: {union_count}개")
    
    
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-12 13:53:18.305857
num_set_df1 : 
254
num_set_df1 : 
254
num_set_df1 : 
254
카테고리 개수가 정확히 254개입니다.
종료: 2024-03-12 13:54:39.823135


In [50]:
print(loaded_unique_subjects- union_set)
print(union_set - loaded_unique_subjects)

set()
set()


중간 저장

In [ ]:
four_1.to_csv('./final/four/final_four_1.txt', sep='\t', index=False)

In [ ]:
four_2.to_csv('./final/four/final_four_2.txt', sep='\t', index=False)

In [ ]:
four_3.to_csv('./final/four/final_four_3.txt', sep='\t', index=False)

불러와서 병합하여 저장

In [2]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
four_1 = pd.read_csv('./final/four/final_four_1.txt', sep='\t')
four_2 = pd.read_csv('./final/four/final_four_2.txt', sep='\t')
four_3 = pd.read_csv('./final/four/final_four_3.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-13 09:19:10.894212
종료: 2024-03-13 09:42:45.530890


드롭한 행이 있으니 cate_n 다시

In [3]:
now = datetime.datetime.now() 
print('시작:', now)

four_1.drop(['cate_n'], axis=1, inplace=True)

four_1['cate_n'] = four_1.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())

#컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id',  'pubyear', 'ref_n', 'refs_uid', 'cate_n', 'subject_traditional', 'within_core']

four_11 = four_1[new_order]
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-13 10:00:53.553263
종료: 2024-03-13 10:27:16.594636


In [4]:
now = datetime.datetime.now() 
print('시작:', now)

four_2.drop(['cate_n'], axis=1, inplace=True)

four_2['cate_n'] = four_2.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())

#컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id',  'pubyear', 'ref_n', 'refs_uid', 'cate_n', 'subject_traditional', 'within_core']

four_22 = four_2[new_order]
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-13 10:27:16.600822
종료: 2024-03-13 11:08:10.983082


In [5]:
now = datetime.datetime.now() 
print('시작:', now)

four_3.drop(['cate_n'], axis=1, inplace=True)

four_3['cate_n'] = four_3.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())

#컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id',  'pubyear', 'ref_n', 'refs_uid', 'cate_n', 'subject_traditional', 'within_core']

four_33 = four_3[new_order]
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-13 11:08:10.988052
종료: 2024-03-13 11:28:20.812560


In [6]:
four_11.to_csv('./final/four/final_four_11.txt', sep='\t', index=False)

In [7]:
four_22.to_csv('./final/four/final_four_22.txt', sep='\t', index=False)

In [8]:
four_33.to_csv('./final/four/final_four_33.txt', sep='\t', index=False)

In [9]:

chunk_files = ['./final/four/final_four_11.txt','./final/four/final_four_22.txt', './final/four/final_four_33.txt' ]

# 병합된 파일 생성
output_file = './final/four.txt'

with open(output_file, 'w', newline='') as f_out:
    for i, file in enumerate(chunk_files):
        with open(file, 'r', newline='') as f_in:
            if i == 0:
                # 첫 번째 파일의 경우, 모든 내용을 복사
                f_out.write(f_in.read())
            else:
                # 두 번째 파일부터는 첫 줄(헤더)을 건너뛰고 나머지 내용을 복사
                next(f_in)  # 첫 줄(헤더) 건너뛰기
                f_out.write(f_in.read())

# =================================

최종파일 불러와서 검사는 메모리 에러..

In [10]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)

four_ = pd.read_csv('./final/four.txt', sep='\t')


now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-13 13:18:20.883573


MemoryError: Unable to allocate 50.7 GiB for an array with shape (5, 1360354933) and data type int64

In [ ]:
now = datetime.datetime.now() 
print('시작:', now)

# 'subject_traditional' 컬럼의 고유한 값들을 찾습니다.


unique_subjects = four['subject_traditional'].unique()


# 합집합의 요소 개수 세기
final_count = len(unique_subjects)



# 254개인지 확인
if final_count == 254:
    print("카테고리 개수가 정확히 254개입니다.")
else:
    print(f"카테고리 개수가 예상과 다릅니다: {final_count}개")
    
    
now = datetime.datetime.now()
print('종료:', now)